# ApplyVortex: Qwen 2.5-7B Fine-Tuning (Kaggle Edition)

This notebook is optimized for **Kaggle T4 or P100 GPUs**. It handles the 30GB RAM limit and correctly locates datasets in `/kaggle/input/`.

In [ ]:
%%capture
# 1. RAM-SAFE INSTALLATION
# Using --no-deps prevents pip from crashing the 30GB RAM limit
!pip install --no-deps --no-cache-dir xformers==0.0.26.post1 trl peft accelerate bitsandbytes
!pip install --no-deps --no-cache-dir "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

### **IMPORTANT**: After the cell above finishes, go to the top menu and select **Run -> Restart Session**. Then continue below.

In [ ]:
from unsloth import FastLanguageModel
import torch
import os

MAX_SEQ_LENGTH = 8192 # Supporting long CVs
dtype = None # Auto detection
load_in_4bit = True # 4-bit quantization reduces VRAM usage significantly

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank 16 is stable for 16GB VRAM GPUs
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✅ Model and Adapters loaded successfully!")

In [ ]:
import json
from datasets import load_dataset

# AUTO-DETECT DATASET ON KAGGLE
# Assumes you uploaded the .jsonl file as a Kaggle dataset
dataset_path = None
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.jsonl'):
            dataset_path = os.path.join(dirname, filename)
            print(f"Found dataset at: {dataset_path}")
            break

if not dataset_path:
    raise FileNotFoundError("Please upload your 'applyvortex_qwen_train.jsonl' file to Kaggle Dataset first.")

dataset = load_dataset("json", data_files=dataset_path, split="train")
print(f"🚀 Loaded {len(dataset)} training examples.")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Can set to True for faster training with short sequences
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60, # Increase this for full training (e.g., 300+)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()
print("🎉 Training Complete!")

In [ ]:
# SAVE LOCALLY TO KAGGLE WORKING DIRECTORY
model.save_pretrained("applyvortex_qwen_adapter") 
tokenizer.save_pretrained("applyvortex_qwen_adapter")
print("💾 Adapter saved to /kaggle/working/applyvortex_qwen_adapter")